In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("/Users/starboy/Documents/Projects/Bank_loan_status/Cleaned_data.csv")

In [3]:
df.head()

,loan_id,customer_id,loan_status,current_loan_amount,term,credit_score,annual_income,years_in_current_job,home_ownership,purpose,monthly_debt,years_of_credit_history,months_since_last_delinquent,number_of_open_accounts,number_of_credit_problems,current_credit_balance,maximum_open_credit,bankruptcies,tax_liens
0,14dd8831-6af5-400b-83ec-68e61888a048,981165ec-3274-42f5-a3b4-d104041a9ca9,Fully Paid,445412.0,Short Term,709.0,1167493.0,8 years,Home Mortgage,Home Improvements,5214.74,17.2,32.0,6.0,1.0,228190.0,416746.0,1.0,0.0
1,4771cc26-131a-45db-b5aa-537ea4ba5342,2de017a3-2e01-49cb-a581-08169e83be29,Fully Paid,262328.0,Short Term,724.0,1174162.0,10+ years,Home Mortgage,Debt Consolidation,33295.98,21.1,8.0,35.0,0.0,229976.0,850784.0,0.0,0.0
2,4eed4e6a-aa2f-4c91-8651-ce984ee8fb26,5efb2b2b-bf11-4dfd-a572-3761a2694725,Fully Paid,99999999.0,Short Term,741.0,2231892.0,8 years,Own Home,Debt Consolidation,29200.53,14.9,29.0,18.0,1.0,297996.0,750090.0,0.0,0.0
3,77598f7b-32e7-4e3b-a6e5-06ba0d98fe8a,e777faab-98ae-45af-9a86-7ce5b33b1011,Fully Paid,347666.0,Long Term,721.0,806949.0,3 years,Own Home,Debt Consolidation,8741.90,12.0,32.0,9.0,0.0,256329.0,386958.0,0.0,0.0
4,d4062e70-befa-4995-8643-a0de73938182,81536ad9-5ccf-4eb8-befb-47a4d608658e,Fully Paid,176220.0,Short Term,724.0,1174162.0,5 years,Rent,Debt Consolidation,20639.70,6.1,32.0,15.0,0.0,253460.0,427174.0,0.0,0.0


#### **1. Debt-To-Income Ratio (DTI)**

In [4]:
df['debt_to_income_ratio'] = df.monthly_debt / (df.annual_income / 12)

#### **2. Credit Utilization Ratio**

In [5]:
df['credit_utilization'] = df.current_credit_balance / df.maximum_open_credit.replace(0,np.nan)

#### **3. Loan-To-Income Ratio**

In [6]:
df['loan_to_income_ratio'] = df.current_loan_amount / df.annual_income.replace(0,np.nan)

**Replace infinities and NaNs with 0 for ratios**

In [7]:
for col in ['debt_to_income_ratio','credit_utilization','loan_to_income_ratio']:
    df[col]= df[col].replace([np.inf,-np.inf],np.nan).fillna(0)

#### **4.Risk Score (0-100 scale)**

In [8]:
def calc_risk(row):
    score = 0
    if row['credit_score'] < 600: score += 50
    elif row['credit_score'] < 700: score += 30
    if row['debt_to_income_ratio'] > 0.4: score += 30
    score += row['number_of_credit_problems'] * 15
    score += row['bankruptcies'] * 25
    return max(0, min(100, score))

In [9]:
df['risk_score']=df.apply(calc_risk,axis=1)

#### **5.Customer Segment**

In [10]:
def seg(row):
    if row['annual_income'] >= 100000 and row['credit_score'] >= 750:
        return 'Premium'
    if row['risk_score'] >= 70:
        return 'High Risk'
    if row['annual_income'] >= 50000 and row['credit_score'] >= 650:
        return 'Standard'
    return 'Subprime'


In [11]:
df['customer_segment']=df.apply(seg,axis=1)

Getting realisting loan amounts

In [12]:
df_clean=df.copy()

In [13]:
realistic_loans= df_clean[df_clean['current_loan_amount']<10000000]['current_loan_amount']
median_loan=realistic_loans.median()

In [14]:
df_clean.loc[df_clean['current_loan_amount']== 99999999,'current_loan_amount'] = median_loan

In [15]:
df_clean['loan_to_income_ratio'] = df_clean['current_loan_amount'] / df_clean['annual_income']

In [16]:
print(f"Fixed {(df['current_loan_amount'] == 99999999).sum()} placeholder loan amounts")

Fixed 11484 placeholder loan amounts


In [17]:
df_clean.to_csv('clean_for_excel.csv',index=False)